In [32]:
import pandas as pd
import copy
import re

In [149]:
a = pd.read_csv('/home/shan/Desktop/HealthLLM_Eval/test_outputs/health_1cot/health_advice_struc_c2_v2_tb.csv')
b = pd.read_csv('/home/shan/Desktop/HealthLLM_Eval/test_outputs/health_1cot/health_advice_struc_c2_v2_tb1.csv')
# c = pd.read_csv('/home/shan/Desktop/HealthLLM_Eval/test_outputs/health_0shot/health_struc_0shot_gpt4_2.csv')
# d = pd.read_csv('/home/shan/Desktop/HealthLLM_Eval/test_outputs/health_0shot/health_struc_0shot_gpt4_3.csv')
# e = pd.read_csv('/home/shan/Desktop/HealthLLM_Eval/test_outputs/health_0shot/health_struc_0shot_gpt4_4.csv')

z = pd.concat([a,b]) #concatenate two dataframes
# z.to_csv('/home/shan/Desktop/HealthLLM_Eval/test_outputs/health_0shot/health_struc_0shot_gpt4.csv', index=False)

In [44]:
# PATH = '/home/shan/Desktop/HealthLLM_Eval/test_outputs/health_1shot/health_advice_struc_c2_v2_tb.csv'
# PATH = '/home/shan/Desktop/HealthLLM_Eval/test_outputs/gpt4_3cot/health_diss_yy_3cot_gpt4.csv'
PATH = '/home/shan/Desktop/HealthLLM_Eval/test_outputs/health_3cot/health_diss_yy_3cot.csv'
# PATH = '/home/shan/Desktop/HealthLLM_Eval/test_outputs/pubmed_1cot/pudmed_c3_v1_tb.csv'
PATH = '/home/shan/Desktop/HealthLLM_Eval/test_outputs/pubmed_0shot/pubmed_0shot_gpt4.csv'
df = pd.read_csv(PATH)

In [45]:
df['answer_list'] = 100

df = df.rename(columns={"output": "direct_output"})
#df.direct_output.values

#pd.set_option('display.max_colwidth', None)
direct_output = copy.deepcopy(df.direct_output)
# setting all answers to be 100 for easier check later
df = pd.DataFrame({'direct_output': direct_output, 'answer_list': [100 for i in range(len(df.context))], 'context_list': df.context, 'y_true': df.label, 'fold': df.fold})
answer_list = df.direct_output

### extract for health advice

In [35]:
answer_list = [str(i).lower() for i in answer_list]
for index, i in enumerate(answer_list):

    if 'no health advice' in i:
        answer_list[index] = '0'
    elif '0) no advice' in i:
        answer_list[index] = '0'
    elif '1) weak advice.' in i:
        answer_list[index] = '0'
    elif 'weak' in i:
        answer_list[index] = '1' 
    elif 'strong' in i:
        answer_list[index] = '2'
    elif 'strong health advice' in i:
        answer_list[index] = '2' 
    elif 'no relationship' in i:
        answer_list[index] = '0'
    elif 'conditionally causative' in i:
        answer_list[index] = '2'
    elif 'causative relationship' in i:
        answer_list[index] ='3'
    elif 'directly correlative' in i:
        answer_list[index] = '1'
    elif 'no advice' in i:
        answer_list[index] = '0'
    elif 'not advice' in i:
        answer_list[index] = '0'
    elif 'weak advice' in i:
        answer_list[index] = '1'
    elif 'strong advice' in i:
        answer_list[index] = '2'
    elif 'no' in i:
        answer_list[index] = '0'
    elif 'none' in i:
        answer_list[index] = '0'

def extract_answer(answer_list):
    return [int(re.findall(r'\d', i)[0]) for i in answer_list]
answer_list = extract_answer(answer_list)
df['answer_list'] = answer_list

In [36]:
b = df.answer_list

In [37]:
a = df.answer_list
df

,direct_output,answer_list,context_list,y_true,fold
0,no medical advice,0,Our data do not indicate a consistent effect o...,0,0
1,weak medical advice,1,"Under careful monitoring, chemotherapy dosage ...",1,0
2,no medical advice,0,Cancer 2017;123:1989-1997.√Ç¬© 2017 American C...,0,0
3,no medical advice,0,The question-behavior effect can be used to re...,1,0
4,weak medical advice,1,Music training program helps appreciation of m...,1,0
...,...,...,...,...,...
4779,no medical advice,0,A higher weight gain contributed in a negative...,0,3
4780,no medical advice,0,Hypertension prevalence increased with age and...,0,3
4781,strong medical advice,2,Continuous monitoring of artemisinin effective...,2,3
4782,no medical advice,0,Our findings suggest that tension and post-sur...,0,3


In [40]:
from sklearn.metrics import classification_report

# Create dictionaries to store the classification report results
dictionary1 = {}
dictionary2 = {}
dictionary3 = {}
dictionary4 = {}

# Separate the original DataFrame into four separate DataFrames based on the 'fold' column
df_fold0 = df[df['fold'] == 0]
df_fold1 = df[df['fold'] == 1]
df_fold2 = df[df['fold'] == 2]
df_fold3 = df[df['fold'] == 3]

# Define the y_true column name and the answer_list column name
y_true_col = 'y_true'
answer_list_col = 'answer_list'

# Loop through each of the four DataFrames, run the classification report, and store the results in the appropriate dictionary
for i, df_fold in enumerate([df_fold0, df_fold1, df_fold2, df_fold3]):
    # Get the fold number
    fold_num = i+1
    
    # Get the y_true and answer_list columns from the DataFrame
    y_true = df_fold[y_true_col]
    answer_list = df_fold[answer_list_col]
    
    # Run the classification report and store the results in the appropriate dictionary
    classification_results = classification_report(y_true, answer_list, output_dict=True, zero_division=1)
    if fold_num == 1:
        dictionary1 = classification_results
    elif fold_num == 2:
        dictionary2 = classification_results
    elif fold_num == 3:
        dictionary3 = classification_results
    elif fold_num == 4:
        dictionary4 = classification_results


In [41]:
# Initialize the accumulators
macro_precision_total = 0
macro_recall_total = 0
macro_f1_total = 0
accuracy_total = 0

macro_precision_total += dictionary1['macro avg']['precision'] + dictionary2['macro avg']['precision'] + dictionary3['macro avg']['precision'] + dictionary4['macro avg']['precision']
macro_recall_total += dictionary1['macro avg']['recall'] + dictionary2['macro avg']['recall'] + dictionary3['macro avg']['recall'] + dictionary4['macro avg']['recall']
macro_f1_total += dictionary1['macro avg']['f1-score'] + dictionary2['macro avg']['f1-score'] + dictionary3['macro avg']['f1-score'] + dictionary4['macro avg']['f1-score']
accuracy_total += dictionary1['accuracy'] + dictionary2['accuracy'] + dictionary3['accuracy'] + dictionary4['accuracy']

macro_precision = macro_precision_total / 4
macro_recall = macro_recall_total / 4
macro_f1 = macro_f1_total / 4
accuracy_avg = accuracy_total / 4

# Print the results
print(f'=============== Eval results for {PATH} ================')
print(f"Precision avg: {macro_precision}")
print(f"Recall avg: {macro_recall}")
print(f"Macro-F1 avg: {macro_f1}")
print(f"Acc avg: {accuracy_avg}")

=============== Eval results for /home/shan/Desktop/HealthLLM_Eval/test_outputs/health_0shot/health_struc_0shot_gpt4.csv ================
Precision avg: 0.6970132435549095
Recall avg: 0.5223192076787182
Macro-F1 avg: 0.516880452319237
Acc avg: 0.6344063545150501


### extraction for causal language

In [46]:
answer_list = [str(i).lower() for i in answer_list]
for index, i in enumerate(answer_list):
    if '0) none' in i:
        answer_list[index] = '0' 
    elif '1) correlational' in i:
        answer_list[index] = '1' 
    elif '2) conditional causal' in i:
        answer_list[index] = '2'
    elif '3) direct causal' in i:
        answer_list[index] = '3'   
    elif 'no claim' in i:
        answer_list[index] = '0'
    elif 'no relation' in i:
        answer_list[index] = '0'
    elif '- direct causation' in i:
        answer_list[index] = '3'
    elif '- correlation' in i:
        answer_list[index] = '1' 
    elif 'conditional causation' in i:
        answer_list[index] = '2'
    elif 'conditionally causative' in i:
        answer_list[index] = '2'
    elif 'causation' in i:
        answer_list[index] = '3'
    elif 'causative relationship' in i:
        answer_list[index] ='3'
    elif '2 - comparison' in i:
        answer_list[index] = '1'
    elif 'directly correlative' in i:
        answer_list[index] = '1'
    elif 'no advice' in i:
        answer_list[index] = '0'
    elif 'not advice' in i:
        answer_list[index] = '0'
    elif 'weak advice' in i:
        answer_list[index] = '1'
    elif 'strong advice' in i:
        answer_list[index] = '2'
    elif 'none' in i:
        answer_list[index] = '0'
    else:
        answer_list[index] = '0'


In [47]:
def extract_answer(answer_list):
    return [int(re.findall(r'\d', i)[0]) for i in answer_list]
answer_list = extract_answer(answer_list)
df['answer_list'] = answer_list

In [48]:
from IPython.display import display

dff_error_check = df[df['answer_list']== 100]
#dff_error_check.display

# set the maximum number of rows to display
pd.set_option('display.max_rows', len(df))

# display the dataframe
df.head()

,direct_output,answer_list,context_list,y_true,fold
0,2 - conditional causation,2,A McMahon score of at least 6 calculated on ad...,0,0
1,0 - no relationship,0,The rs7903146 (C/T) polymorphism of the TCF7L2...,1,0
2,1 - correlation,1,A fifth of patients with a negative CDS of the...,1,0
3,1 - correlation,1,We observed an association between EH/EC and a...,1,0
4,1 - correlation,1,The CHA2DS2-VASc tool predicts thromboembolic ...,1,0


In [49]:
from sklearn.metrics import classification_report

# Create dictionaries to store the classification report results
dictionary1 = {}
dictionary2 = {}
dictionary3 = {}
dictionary4 = {}

# Separate the original DataFrame into four separate DataFrames based on the 'fold' column
df_fold0 = df[df['fold'] == 0]
df_fold1 = df[df['fold'] == 1]
df_fold2 = df[df['fold'] == 2]
df_fold3 = df[df['fold'] == 3]

# Define the y_true column name and the answer_list column name
y_true_col = 'y_true'
answer_list_col = 'answer_list'

# Loop through each of the four DataFrames, run the classification report, and store the results in the appropriate dictionary
for i, df_fold in enumerate([df_fold0, df_fold1, df_fold2, df_fold3]):
    # Get the fold number
    fold_num = i+1
    
    # Get the y_true and answer_list columns from the DataFrame
    y_true = df_fold[y_true_col]
    answer_list = df_fold[answer_list_col]
    
    # Run the classification report and store the results in the appropriate dictionary
    classification_results = classification_report(y_true, answer_list, output_dict=True)
    if fold_num == 1:
        dictionary1 = classification_results
    elif fold_num == 2:
        dictionary2 = classification_results
    elif fold_num == 3:
        dictionary3 = classification_results
    elif fold_num == 4:
        dictionary4 = classification_results


In [50]:
# Initialize the accumulators
macro_precision_total = 0
macro_recall_total = 0
macro_f1_total = 0
accuracy_total = 0

macro_precision_total += dictionary1['macro avg']['precision'] + dictionary2['macro avg']['precision'] + dictionary3['macro avg']['precision'] + dictionary4['macro avg']['precision']
macro_recall_total += dictionary1['macro avg']['recall'] + dictionary2['macro avg']['recall'] + dictionary3['macro avg']['recall'] + dictionary4['macro avg']['recall']
macro_f1_total += dictionary1['macro avg']['f1-score'] + dictionary2['macro avg']['f1-score'] + dictionary3['macro avg']['f1-score'] + dictionary4['macro avg']['f1-score']
accuracy_total += dictionary1['accuracy'] + dictionary2['accuracy'] + dictionary3['accuracy'] + dictionary4['accuracy']

macro_precision = macro_precision_total / 4
macro_recall = macro_recall_total / 4
macro_f1 = macro_f1_total / 4
accuracy_avg = accuracy_total / 4

# Print the results
print(f'=============== Eval results for {PATH} ================')
print(f"Precision avg: {macro_precision}")
print(f"Recall avg: {macro_recall}")
print(f"Macro-F1 avg: {macro_f1}")
print(f"Acc avg: {accuracy_avg}")

=============== Eval results for /home/shan/Desktop/HealthLLM_Eval/test_outputs/pubmed_0shot/pubmed_0shot_gpt4.csv ================
Precision avg: 0.3458183674859787
Recall avg: 0.3928914041417187
Macro-F1 avg: 0.30391969281652226
Acc avg: 0.36630390017436326


# BoW

In [14]:
import pandas as pd
P = '/home/shan/Desktop/HealthLLM_Eval/data/'

health_advice_diss = pd.read_csv(P+'advice_discussion_annotation.csv')
health_advice_un = pd.read_csv(P+'advice_unstructured_abs_annotation.csv')
health_advice = pd.read_csv(P+'advice_structured_abs_annotation.csv')
# press_release = pd.read_csv('press_release_causal_language_annotation.csv')
pubmed_causal = pd.read_csv(P+'pubmed_causal_language_annotation.csv')
health_advice_diss['type'] = ['diss' for i in health_advice_diss.label]
health_advice_un['type'] = ['un' for i in health_advice_un.label]
health_advice['type'] = ['struc' for i in health_advice.label]
health = pd.concat([health_advice_diss, health_advice_un, health_advice]) 

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    health.sentence, health.label, test_size=0.01, random_state=42)

In [16]:
X_test, y_test = df.context_list, df.answer_list

In [17]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=7e-7, random_state=42,
                          max_iter=50000, tol=None)),
])


In [18]:
from sklearn import metrics
text_clf.fit(X_train, y_train)
predicted = text_clf.predict(X_test)
print(metrics.classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.93      0.96      0.95      2837
           1       0.30      0.27      0.29       153
           2       0.60      0.34      0.44       155

    accuracy                           0.90      3145
   macro avg       0.61      0.53      0.56      3145
weighted avg       0.89      0.90      0.89      3145



In [19]:
c = predicted

## BERT


In [10]:
import pandas as pd

P = '/home/shan/Desktop/HealthLLM_Eval/data/'

health_advice_diss = pd.read_csv(P+'advice_discussion_annotation.csv')
health_advice_un = pd.read_csv(P+'advice_unstructured_abs_annotation.csv')
health_advice = pd.read_csv(P+'advice_structured_abs_annotation.csv')
# press_release = pd.read_csv('press_release_causal_language_annotation.csv')
pubmed_causal = pd.read_csv(P+'pubmed_causal_language_annotation.csv')
health_advice_diss['type'] = ['diss' for i in health_advice_diss.label]
health_advice_un['type'] = ['un' for i in health_advice_un.label]
health_advice['type'] = ['struc' for i in health_advice.label]
health = pd.concat([health_advice_diss, health_advice_un, health_advice]) 

In [11]:
# PATH = P+'test_data_4folds/health_advice_discuss_annotation_test_folds.csv'
# PATH = P+'test_data_4folds/health_advice_unsturctured_abs_annotation_test_folds.csv' 
# PATH = P+'test_data_4folds/health_advice_sturctured_abs_annotation_test_folds.csv'
PATH = P+'test_data_4folds/pubmed_causal_annotation_test_folds.csv'

SIZE = 0.2
train_df = pubmed_causal
# train_df = health 
train_df = train_df.sample(frac=SIZE).reset_index(drop=True)
val_df = pd.read_csv(PATH)

In [12]:
747+3145+759+4784

9435

In [13]:
val_df

,sentence_id,label,sentence,fold
0,1273,0,A McMahon score of at least 6 calculated on ad...,0
1,2734,1,The rs7903146 (C/T) polymorphism of the TCF7L2...,0
2,2275,1,A fifth of patients with a negative CDS of the...,0
3,2717,1,We observed an association between EH/EC and a...,0
4,2109,1,The CHA2DS2-VASc tool predicts thromboembolic ...,0
...,...,...,...,...
2441,619,0,Current chemoprevention guidelines may be fall...,3
2442,1958,2,"Subsequently, the aggregate may have had a syn...",3
2443,1556,3,Overnight closed-loop therapy resulted in bett...,3
2444,1023,0,"""The minimum prevalence of HNF1A-MODY among di...",3


In [14]:
from datasets import Dataset, DatasetDict
tds = Dataset.from_pandas(train_df)
vds = Dataset.from_pandas(val_df)

In [15]:
dataset = DatasetDict()

dataset['train'] = tds
dataset['validation'] = vds

In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence_id', 'label', 'sentence'],
        num_rows: 123
    })
    validation: Dataset({
        features: ['sentence_id', 'label', 'sentence', 'fold'],
        num_rows: 2446
    })
})

In [17]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments
from transformers import Trainer



# Load the Biobert tokenizer
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.2")

# Load the Biobert model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained("dmis-lab/biobert-base-cased-v1.2", num_labels=4)
# model = torch.compile(model)
dataset = dataset.map(lambda x: tokenizer(x['sentence'], padding='max_length', truncation=True, max_length=128), batched=True)

# Define the training arguments for the Trainer
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=200,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=1024,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    evaluation_strategy="epoch",
    bf16=True,
    torch_compile=True, # optimizations
    optim="adamw_torch_fused"
)

# Define the Trainer object and train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    tokenizer=tokenizer
)
trainer.train()

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.2 were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

Epoch,Training Loss,Validation Loss
1,No log,1.402547
2,No log,1.402074
3,No log,1.400951
4,No log,1.399405
5,No log,1.397168
6,No log,1.394465
7,No log,1.390953
8,No log,1.386882
9,No log,1.382123
10,No log,1.376541


TrainOutput(global_step=200, training_loss=0.48522635608911513, metrics={'train_runtime': 311.2118, 'train_samples_per_second': 79.046, 'train_steps_per_second': 0.643, 'total_flos': 1618162047590400.0, 'train_loss': 0.48522635608911513, 'epoch': 200.0})

In [18]:
predictions = trainer.predict(dataset['validation'])
predicted_labels = predictions.predictions.argmax(axis=-1).tolist()

In [19]:
df = val_df

In [20]:
df

,sentence_id,label,sentence,fold
0,1273,0,A McMahon score of at least 6 calculated on ad...,0
1,2734,1,The rs7903146 (C/T) polymorphism of the TCF7L2...,0
2,2275,1,A fifth of patients with a negative CDS of the...,0
3,2717,1,We observed an association between EH/EC and a...,0
4,2109,1,The CHA2DS2-VASc tool predicts thromboembolic ...,0
...,...,...,...,...
2441,619,0,Current chemoprevention guidelines may be fall...,3
2442,1958,2,"Subsequently, the aggregate may have had a syn...",3
2443,1556,3,Overnight closed-loop therapy resulted in bett...,3
2444,1023,0,"""The minimum prevalence of HNF1A-MODY among di...",3


In [21]:
df['answer_list'] = predicted_labels
d = predicted_labels 

In [22]:
from sklearn.metrics import classification_report

# Create dictionaries to store the classification report results
dictionary1 = {}
dictionary2 = {}
dictionary3 = {}
dictionary4 = {}

# Separate the original DataFrame into four separate DataFrames based on the 'fold' column
df_fold0 = df[df['fold'] == 0]
df_fold1 = df[df['fold'] == 1]
df_fold2 = df[df['fold'] == 2]
df_fold3 = df[df['fold'] == 3]

# Define the y_true column name and the answer_list column name
y_true_col = 'label'
answer_list_col = 'answer_list'

# Loop through each of the four DataFrames, run the classification report, and store the results in the appropriate dictionary
for i, df_fold in enumerate([df_fold0, df_fold1, df_fold2, df_fold3]):
    # Get the fold number
    fold_num = i+1
    
    # Get the y_true and answer_list columns from the DataFrame
    y_true = df_fold[y_true_col]
    answer_list = df_fold[answer_list_col]
    
    # Run the classification report and store the results in the appropriate dictionary
    classification_results = classification_report(y_true, answer_list, output_dict=True, zero_division=1)
    if fold_num == 1:
        dictionary1 = classification_results
    elif fold_num == 2:
        dictionary2 = classification_results
    elif fold_num == 3:
        dictionary3 = classification_results
    elif fold_num == 4:
        dictionary4 = classification_results


In [23]:
# Initialize the accumulators
macro_precision_total = 0
macro_recall_total = 0
macro_f1_total = 0
accuracy_total = 0

macro_precision_total += dictionary1['macro avg']['precision'] + dictionary2['macro avg']['precision'] + dictionary3['macro avg']['precision'] + dictionary4['macro avg']['precision']
macro_recall_total += dictionary1['macro avg']['recall'] + dictionary2['macro avg']['recall'] + dictionary3['macro avg']['recall'] + dictionary4['macro avg']['recall']
macro_f1_total += dictionary1['macro avg']['f1-score'] + dictionary2['macro avg']['f1-score'] + dictionary3['macro avg']['f1-score'] + dictionary4['macro avg']['f1-score']
accuracy_total += dictionary1['accuracy'] + dictionary2['accuracy'] + dictionary3['accuracy'] + dictionary4['accuracy']

macro_precision = macro_precision_total / 4
macro_recall = macro_recall_total / 4
macro_f1 = macro_f1_total / 4
accuracy_avg = accuracy_total / 4

# Print the results
print(f'=============== BERT Eval results for {PATH} using {SIZE*100}% data ================')
print(f"Precision avg: {macro_precision}")
print(f"Recall avg: {macro_recall}")
print(f"Macro-F1 avg: {macro_f1}")
print(f"Acc avg: {accuracy_avg}")

=============== BERT Eval results for /home/shan/Desktop/HealthLLM_Eval/data/test_data_4folds/pubmed_causal_annotation_test_folds.csv using 20.0% data ================
Precision avg: 0.7358248640678164
Recall avg: 0.569699061949458
Macro-F1 avg: 0.5971158446821065
Acc avg: 0.7199524245049902


# figure

In [1]:
import numpy as np
from sklearn.metrics import f1_score
from scipy import stats
import itertools
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
a = [ 0.648, 0.712, 0.77, 0.682]
b = [ 0.671, 0.670, 0.718, 0.649]
c = [ 0.602, 0.640, 0.753, 0.675]
d = [ 0.800, 0.821, 0.902, 0.851]

In [3]:
y_preds = [a, b, c, d]

In [4]:
y_true = df.label

NameError: name 'df' is not defined

In [5]:
f1_score(y_true, a, average='macro'), f1_score(y_true, b, average='macro'), f1_score(y_true, c, average='macro'), f1_score(y_true, d, average='macro')

NameError: name 'y_true' is not defined

In [45]:
x1 = f1_score(y_true, models[0], average=None)

In [46]:
x2 = f1_score(y_true, models[3], average=None)

In [48]:
x1, x2

(array([0.94979599, 0.45604396, 0.56055363]),
 array([0.97639708, 0.65408805, 0.75238095]))

In [47]:
print(np.var(x1), np.var(x2))

0.04513588414900216 0.018191981710584577


In [6]:
import numpy as np
from scipy.stats import ttest_ind 

models = [a, b, c, d]
# initialize an empty matrix to store the F1 scores
f1_matrix = np.zeros((4,4))

# compute F1 scores for all pairs of models
for i in range(len(models)):
    for j in range(i+1, len(models)):
        f1 = models[i]
        f2 = models[j]
        # f1 = f1_score(y_true, y_pred1, average=None)
        # f2 = f1_score(y_true, y_pred2, average=None)
        t_statistic, p_value = ttest_ind(f1, f2, equal_var=True)
        f1_matrix[i,j] = p_value

# print the matrix
print(f1_matrix)


[[0.00000000e+00 4.15030067e-01 4.22831067e-01 6.17279803e-03]
 [0.00000000e+00 0.00000000e+00 7.96916609e-01 7.56314317e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 4.06857629e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]]


In [ ]:
ttest_ind(a, b), ttest_ind(a, c), ttest_ind(a, d), ttest_ind(b, c), ttest_ind(b, d), ttest_ind(c, d)
ttest_rel(a, b), ttest_rel(a, c), ttest_rel(a, d), ttest_rel(b, c), ttest_rel(b, d), ttest_rel(c, d)